In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
import pandas as pd
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix, roc_auc_score

In [2]:
model = keras.models.load_model("../Saved Data/Model")
X_test = np.load('../Saved Data/Test Set/X_test.npy')
y_test = np.load('../Saved Data/Test Set/y_test.npy')

In [3]:
probabilities_vector = model.predict(X_test)
y_pred = np.argmax(probabilities_vector, axis=1)
np.savetxt('../Saved Data/Predicted Labels/python_model.txt', y_pred, fmt='%d')

26/26 [==============================] - 0s 6ms/step


In [4]:
accuracy_score = accuracy_score(y_test, y_pred)
precision_score = precision_score(y_test, y_pred, average='macro')
recall_score = recall_score(y_test, y_pred, average='macro')
f1_score = f1_score(y_test, y_pred, average='macro')
confusion_matrix = confusion_matrix(y_test, y_pred)
classification_report = classification_report(y_test, y_pred)
# AUC_score = roc_auc_score(y_test, probabilities_vector, multi_class='ovo')

In [5]:
if os.path.exists('../Saved Data/Temp Data/python_model.json'):
    os.remove('../Saved Data/Temp Data/python_model.json')

In [6]:
data = {"accuracy": accuracy_score, "precision": precision_score, "recall": recall_score, "f1": f1_score}
json_data = json.dumps(data, indent = 4)
json_file = open('../Saved Data/Temp Data/python_model.json', 'w')
json_file.write(json_data)
print(f"Classification Accuracy = {round(accuracy_score * 100, 2)}%\nPrecision = {round(precision_score * 100, 2)}%\nRecall = {round(recall_score * 100, 2)}%\nF1 Score = {round(f1_score * 100, 2)}%\n")
json_file.close()

Classification Accuracy = 72.95%
Precision = 81.07%
Recall = 70.27%
F1 Score = 70.8%



In [7]:
print(classification_report)

              precision    recall  f1-score   support

           0       1.00      0.30      0.46       108
           1       0.92      0.68      0.78       112
           2       0.79      0.80      0.80        61
           3       0.79      0.69      0.73       144
           4       0.45      0.92      0.61       143
           5       0.92      0.83      0.87       260

    accuracy                           0.73       828
   macro avg       0.81      0.70      0.71       828
weighted avg       0.82      0.73      0.73       828



In [8]:
print(confusion_matrix)

[[ 32   2   2   3  67   2]
 [  0  76   4   6  21   5]
 [  0   1  49   2   8   1]
 [  0   2   2  99  33   8]
 [  0   1   0   8 131   3]
 [  0   1   5   8  29 217]]
